In [ ]:
import kagglehub
import os
import shutil
from sklearn.model_selection import train_test_split

path_root = r'D:\Mine\Basics\Coding\Python_projects\SimpleAutoML\ml\datasets'
os.makedirs(path_root, exist_ok=True)

In [52]:
def train_val_test_split(X, y, train_size, val_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, shuffle=True, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=val_size/(1 - train_size), shuffle=True, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [57]:
# Download latest butterflies
path = kagglehub.dataset_download("deepakat002/yolov8-segmentation-data-butterfly")
print("Path to dataset files:", path)

path_root_butterfly = os.path.join(path_root, 'Butterfly')
data_root_butterfly = os.path.join(path_root_butterfly, os.listdir(path)[0])

if 'Butterfly' in os.listdir(path_root):
    shutil.rmtree(path_root_butterfly)
else:
    os.makedirs(path_root_butterfly, exist_ok=True)

shutil.copytree(path, path_root_butterfly, dirs_exist_ok=True)

for purp in ['train', 'val']:
    os.makedirs(os.path.join(path_root_butterfly, purp), exist_ok=True)
    for dir in ['images', 'labels']:
        os.makedirs(os.path.join(path_root_butterfly, purp, dir), exist_ok=True)

X_data = []
y_data = []
for obj in os.listdir(data_root_butterfly):
    image_extentions = ['png', 'jpeg', 'jpg']
    if obj.partition('.')[2] in image_extentions:
        X_data.append(obj)
    elif obj.endswith(".txt"):
        y_data.append(obj)
        obj = os.path.join(data_root_butterfly, obj)
        content = []
        with open(obj, "rt", encoding="utf-8") as read:
            for line in read.readlines():
                new_line = line.split(" ")
                new_line[0] = str(0)
                content.append(" ".join(new_line))
        with open(obj, "wt", encoding="utf-8") as write:
            write.writelines(content)

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, train_size=0.7, shuffle=True, random_state=42)

for X, y, purp in ((X_train, y_train, 'train'), (X_val, y_val, 'val')):
    for file, label in zip(X, y):
        shutil.move(os.path.join(data_root_butterfly, file), os.path.join(path_root_butterfly, purp, 'images', file))
        shutil.move(os.path.join(data_root_butterfly, label), os.path.join(path_root_butterfly, purp, 'labels', label))

shutil.rmtree(data_root_butterfly)

with open(os.path.join(path_root_butterfly, 'data.yaml'), 'wt', encoding="utf-8") as conf:
    conf.write(f"train: ../train/images\n")
    conf.write(f"val: ../val/images\n")
    conf.write(f"\n")
    conf.write(f"nc: 1\n")
    conf.write(f"names: ['Butterfly']\n")

Path to dataset files: C:\Users\horo\.cache\kagglehub\datasets\deepakat002\yolov8-segmentation-data-butterfly\versions\3


In [ ]:
# Download the grapes
path = kagglehub.dataset_download("nicolaasregnier/grape-grapes")
print("Path to dataset files:", path)

path_root_grape = os.path.join(path_root, 'Grape')
data_root_grape = os.path.join(path_root_grape, os.listdir(path)[os.listdir(path).index("MyData")])

if 'Grape' in os.listdir(path_root):
    shutil.rmtree(path_root_grape)
else:
    os.makedirs(path_root_grape, exist_ok=True)

shutil.copytree(path, path_root_grape, dirs_exist_ok=True)
os.remove(os.path.join(path_root_grape, "data.yaml"))
for dir in os.listdir(path_root_grape):
    if dir != 'MyData':
        shutil.rmtree(os.path.join(path_root_grape, dir))

for dir in ('images', 'labels'):
    os.makedirs(os.path.join(path_root_grape, dir), exist_ok=True)

types = {"Chardonnay": 0, "PinotGris": 1, "PinotNoir": 2}
for type_grape in os.listdir(data_root_grape):
    for label in os.listdir(os.path.join(data_root_grape, type_grape, 'labels')):
        label = os.path.join(data_root_grape, type_grape, 'labels', label)
        if not os.path.isfile(label):
            continue
        if "classes" in label:
            os.remove(label)
            continue
        content = []
        with open(label, "rt", encoding="utf-8") as read:
            for line in read.readlines():
                new_line = line.split(" ")
                new_line[0] = str(types[type_grape])
                content.append(" ".join(new_line))
        with open(label, "wt", encoding="utf-8") as write:
            write.writelines(content)
        shutil.move(label, os.path.join(path_root_grape, 'labels'))

    for image in os.listdir(os.path.join(data_root_grape, type_grape, 'images')):
        image = os.path.join(data_root_grape, type_grape, 'images', image)
        shutil.move(image, os.path.join(path_root_grape, 'images'))

shutil.rmtree(data_root_grape)

for purp in ['train', 'val']:
    os.makedirs(os.path.join(path_root_grape, purp), exist_ok=True)
    for dir in ['images', 'labels']:
        os.makedirs(os.path.join(path_root_grape, purp, dir), exist_ok=True)

X_data = os.listdir(os.path.join(path_root_grape, 'images'))
y_data = os.listdir(os.path.join(path_root_grape, 'labels'))

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, train_size=0.7, shuffle=True, random_state=42)

for X, y, purp in ((X_train, y_train, 'train'), (X_val, y_val, 'val')):
    for file, label in zip(X, y):
        shutil.move(os.path.join(path_root_grape, 'images', file), os.path.join(path_root_butterfly, purp, 'images', file))
        shutil.move(os.path.join(path_root_grape, 'labels', label), os.path.join(path_root_butterfly, purp, 'labels', label))

for dir in ('images', 'labels'):
    shutil.rmtree(os.path.join(path_root_grape, dir))

with open(os.path.join(path_root_grape, 'data.yaml'), 'wt', encoding="utf-8") as conf:
    conf.write(f"train: ../train/images\n")
    conf.write(f"val: ../val/images\n")
    conf.write(f"\n")
    conf.write(f"nc: 3\n")
    conf.write(f"names: ['Chardonnay', 'PinotGris', 'PinotNoir']\n")

Path to dataset files: C:\Users\horo\.cache\kagglehub\datasets\nicolaasregnier\grape-grapes\versions\27


In [ ]:
# Download the potholes
path = kagglehub.dataset_download("farzadnekouei/pothole-image-segmentation-dataset")
print("Path to dataset files:", path)

path_root_pothole = os.path.join(path_root, 'Pothole')
data_root_pothole = os.path.join(path_root_pothole, os.listdir(path)[0])

if 'Pothole' in os.listdir(path_root):
    shutil.rmtree(path_root_pothole)
else:
    os.makedirs(path_root_pothole, exist_ok=True)

shutil.copytree(path, path_root_pothole, dirs_exist_ok=True)

for dir in ('images', 'labels'):
    os.makedirs(os.path.join(path_root_pothole, dir), exist_ok=True)

for purp in ['train', 'val']:
    os.makedirs(os.path.join(path_root_pothole, purp), exist_ok=True)
    for dir in ['images', 'labels']:
        os.makedirs(os.path.join(path_root_pothole, purp, dir), exist_ok=True)

for purp in ('train', 'valid'):
    for src in ('images', 'labels'):
        for file in os.listdir(os.path.join(data_root_pothole, purp, src)):
            shutil.move(os.path.join(data_root_pothole, purp, src, file), os.path.join(path_root_pothole, src))

shutil.rmtree(data_root_pothole)

X_data = os.listdir(os.path.join(path_root_pothole, 'images'))
y_data = os.listdir(os.path.join(path_root_pothole, 'labels'))

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, train_size=0.7, shuffle=True, random_state=42)

for X, y, purp in ((X_train, y_train, 'train'), (X_val, y_val, 'val')):
    for file, label in zip(X, y):
        shutil.move(os.path.join(path_root_pothole, 'images', file), os.path.join(path_root_pothole, purp, 'images', file))
        shutil.move(os.path.join(path_root_pothole, 'labels', label), os.path.join(path_root_pothole, purp, 'labels', label))

for dir in ('images', 'labels'):
    shutil.rmtree(os.path.join(path_root_pothole, dir))

with open(os.path.join(path_root_pothole, 'data.yaml'), 'wt', encoding="utf-8") as conf:
    conf.write(f"train: ../train/images\n")
    conf.write(f"val: ../val/images\n")
    conf.write(f"\n")
    conf.write(f"nc: 1\n")
    conf.write(f"names: ['Pothole']\n")

Path to dataset files: C:\Users\horo\.cache\kagglehub\datasets\farzadnekouei\pothole-image-segmentation-dataset\versions\2
